Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# PyTorch Pretrained BERT on AzureML with SQuAD Dataset

In this notebook, you will find the following contents:
- Download SQuAD dataset on the remote compute and store them in Azure storage
- Speed-up fine-tuning BERT for SQuAD dataset on AzureML GPU clusters

## Prerequisites
Follow instructions in BERT_pretraining.ipynb notebook for setting up AzureML

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize workspace

To create or access an Azure ML Workspace, you will need to import the AML library and the following information:
* A name for your workspace
* Your subscription id
* The resource group name

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace or create a new one. 

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.setup()
ws_details = ws.get_details()
print('Name:\t\t{}\nLocation:\t{}'
      .format(ws_details['name'],
              ws_details['location']))

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [ ]:
import os
import os.path as path
project_root = path.abspath(path.join(os.getcwd(),"../../../"))

### Define AzureML datastore to collect training dataset

To make data accessible for remote training, AML provides a convenient way to do so via a [Datastore](https://docs.microsoft.com/azure/machine-learning/service/how-to-access-data). The datastore provides a mechanism for you to upload/download data to Azure Storage, and interact with it from your remote compute targets.

Each workspace is associated with a default Azure Blob datastore named `'workspaceblobstore'`. In this work, we use this default datastore to collect the SQuAD training data .

In [ ]:
from azureml.core import Datastore
ds = ws.get_default_datastore()

The data for SQuAD can be downloaded with the following links and should be saved in a blob storage.
- [train-v1.1.json](https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json)
- [dev-v1.1.json](https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json)
- [evaluate-v1.1.py](https://github.com/allenai/bi-att-flow/blob/master/squad/evaluate-v1.1.py)

The following code will upload the training data to the path ./squad on the default datastore.

In [ ]:
import os
ds.upload(src_dir=project_root+'\data\squad', target_path='squad')
ds.upload(src_dir=os.path.join(project_root,'data','bert-large-checkpoints') , target_path='bert-large-checkpoints')

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial. 

In [ ]:
from azureml.core import Experiment

experiment_name = 'BERT-SQuAD'
experiment = Experiment(ws, name=experiment_name)

## Fine-tuning BERT with Distributed Training
As our `SQuAD` dataset are ready in Azure storage, we can start the fine-tune the model by exploting the power of distributed training. 

### Create a GPU remote compute target

We need to create a GPU [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) to perform the fine-tuning. In this example, we create an AmlCompute cluster as our training compute resource.

This code creates a cluster for you if it does not already exist in your workspace.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
gpu_cluster_name = "bertcodetesting"

try:
    gpu_compute_target = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC24s_v3", max_nodes=4)

    # create the cluster
    gpu_compute_target = AmlCompute.create(ws, gpu_cluster_name, compute_config)
    gpu_compute_target.wait_for_completion(show_output=True)

    # Use the 'status' property to get a detailed status for the current cluster. 
    print(gpu_compute_target.status.serialize())

### Create a PyTorch estimator for fine-tuning
Let us create a new PyTorch estimator to run the fine-tuning script `run_squad.py`, that is already provided at [the original repository](https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_squad.py). Please refer [here](https://github.com/huggingface/pytorch-pretrained-BERT#fine-tuning-with-bert-running-the-examples) for more detail about the script. 

The original `run_squad.py` script uses PyTorch distributed launch untility to launch multiple processes across nodes and GPUs. We prepared a modified version [run_squad_azureml.py](./run_squad_azureml.py) so that we can launch it based on AzureML build-in MPI backend.

To use AML's tracking and metrics capabilities, we need to add a small amount of AzureML code inside the training script.

In `run_squad_azureml.py`, we will log some metrics to our AML run. To do so, we will access the AML run object within the script:
```Python
from azureml.core.run import Run
run = Run.get_context()
```
Further within `run_squad_azureml.py`, we log learning rate, training loss and prediction scores the model achieves as:
```Python
run.log('lr', np.float(args.learning_rate))
...

for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")): 
    ...
    run.log('train_loss', np.float(loss))

..
```

In [ ]:
from azureml.train.dnn import PyTorch
from azureml.core.runconfig import RunConfiguration
from azureml.core.container_registry import ContainerRegistry

run_user_managed = RunConfiguration()
run_user_managed.environment.python.user_managed_dependencies = True

# Define custom Docker image info
image_name = 'mcr.microsoft.com/azureml/bert:pretrain-openmpi3.1.2-cuda10.0-cudnn7-ubuntu16.04'

estimator = PyTorch(source_directory='../../../',
                    compute_target=gpu_compute_target,
                     #Docker image
                    use_docker=True,
                    custom_docker_image=image_name,
                    user_managed=True,
                    script_params = {
                          '--bert_model':'bert-large-uncased',
                          "--model_file_location": ds.path('bert-large-checkpoints/').as_mount(),
                          '--model_file': 'bert_encoder_epoch_245.pt',
                          '--do_train' : '',
                          '--do_predict': '',
                          '--train_file': ds.path('squad/train-v1.1.json').as_mount(),
                          '--predict_file': ds.path('squad/dev-v1.1.json').as_mount(),
                          '--max_seq_length': 512,
                          '--train_batch_size': 8,
                          '--learning_rate': 3e-5,
                          '--num_train_epochs': 2.0,
                          '--doc_stride': 128,
                          '--seed': 32,
                          '--gradient_accumulation_steps':4,
                          '--warmup_proportion':0.25,
                          '--output_dir': './outputs',
                          '--fp16':'',
                          #'--loss_scale':128,
                    },
                    entry_script='./finetune/run_squad_azureml.py',
                    node_count=1,
                    process_count_per_node=4,
                    distributed_backend='mpi',
                    use_gpu=True)

# path to the Python environment in the custom Docker image
estimator._estimator_config.environment.python.interpreter_path = '/opt/miniconda/envs/amlbert/bin/python'

### Submit and Monitor your run

In [ ]:
run = experiment.submit(estimator)
from azureml.widgets import RunDetails
RunDetails(run).show()

To achieve over **90.5 F1 score** and **83.5 Exact-Match** with `SQuAD v1.1` dataset, it requires **2** epochs when fine-tune with `BERT large` model. Below please find the elapsed time using deferent Azure GPU VMs and configures. 

The default configuration in this notebook uses 2 `STANDARD_NC24rs_v3` (8 x V100) with `fp16` enabled. The training phase should take **22 mins** to complete 2 epochs. 

|  GPU counts 	|    1 GPU    	|         2 GPU 	| 4 GPU      	| 8 GPU      	|
|------------:	|:-----------:	|--------------:	|------------	|------------	|
| NCv3-series 	|     340 mins  |    180 mins 	    |    80 mins 	|   48 mins 	|
| NCv3 with fp16|     140 mins  |    79 mins 	    |    38 mins 	|   22 mins 	|